# Investigate a potential blob bump on mainnet
See .env.example to set the environment variables.

In [1]:
!pip install jupysql clickhouse_sqlalchemy matplotlib python-dotenv pandas seaborn imageio > /dev/null 2>&1

In [ ]:
import dotenv
import os
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import imageio

dotenv.load_dotenv()

# Load and display config
start_date = os.getenv('START_DATE_TIME')
if not start_date:
    raise ValueError("START_DATE_TIME environment variable is required")

end_date = os.getenv('END_DATE_TIME')
if not end_date:
    end_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")

network = os.getenv('NETWORK')

print(f"start_date: {start_date}")
print(f"end_date: {end_date}")

# Convert start and end dates to datetime objects
try:
    start_dt = datetime.fromisoformat(start_date.replace("Z", "+00:00"))
except ValueError as e:
    raise ValueError(f"Invalid start date format. Date must be in ISO format. Error: {str(e)}")

try:
    if isinstance(end_date, datetime):
        end_dt = end_date
    else:
        end_dt = datetime.fromisoformat(str(end_date).replace("Z", "+00:00"))
except ValueError as e:
    raise ValueError(f"Invalid end date format. Date must be in ISO format. Error: {str(e)}")

# Calculate the difference in hours
hours = (end_dt - start_dt).total_seconds() / 3600

print(f"hours: {hours}")


In [3]:
# Connect to ClickHouse
import os
username = os.getenv('XATU_CLICKHOUSE_USERNAME')
password = os.getenv('XATU_CLICKHOUSE_PASSWORD')
host = os.getenv('XATU_CLICKHOUSE_HOST')


db_url = f"clickhouse+http://{username}:{password}@{host}:443/default?protocol=https"

from sqlalchemy import create_engine, text

engine = create_engine(db_url)

connection = engine.connect()

In [ ]:
# Query for MEV relay delivered slots
block_query = text("""
    SELECT DISTINCT
        slot
    FROM mev_relay_proposer_payload_delivered FINAL
    WHERE
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
    ORDER BY slot ASC
""")

result = connection.execute(block_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')})
mev_slots = pd.DataFrame(result.fetchall(), columns=['slot'])

print(mev_slots)

In [5]:
# Query for all slots in time window
all_slots_query = text("""
    SELECT DISTINCT
        slot,
        slot_start_date_time as time
    FROM beacon_api_eth_v1_events_block FINAL 
    WHERE
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
    ORDER BY slot ASC
""")

result = connection.execute(all_slots_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')})
all_slots = pd.DataFrame(result.fetchall(), columns=['slot', 'time'])


In [ ]:
# Query to get block sizes for all slots
print("Getting block sizes...")
block_sizes_query = text("""
    SELECT 
        slot,
        block_total_bytes_compressed,
        slot_start_date_time as time,
        proposer_index,
        block_root
    FROM canonical_beacon_block FINAL
    WHERE 
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
""")
block_sizes = pd.DataFrame(
    connection.execute(block_sizes_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'block_total_bytes_compressed', 'time', 'proposer_index', 'block_root']
)

print("Getting proposer entities...")
# Get proposer entities
proposer_query = text("""
    SELECT 
        `index` as proposer_index,
        entity
    FROM ethseer_validator_entity
    WHERE 
        meta_network_name = 'mainnet'
""")
proposer_entities = pd.DataFrame(
    connection.execute(proposer_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['proposer_index', 'entity']
)

print("Getting blob sidecar data...")
# Get blob sidecar data for our slots
blob_sidecars_query = text("""
    SELECT 
        slot,
        COUNT(*) as num_blobs,
        SUM(blob_size) as total_blob_size
    FROM canonical_beacon_blob_sidecar FINAL
    WHERE
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
    GROUP BY slot, toDate(slot_start_date_time)
""")
blob_sidecars = pd.DataFrame(
    connection.execute(blob_sidecars_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'num_blobs', 'total_blob_size']
)

print("Getting first seen in p2p data...")
first_seen_in_p2p_query = text("""
    SELECT
        slot,
        block as block_root,
        MIN(propagation_slot_start_diff) as first_seen_in_p2p
    FROM libp2p_gossipsub_beacon_block FINAL
    WHERE
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
    GROUP BY slot, block
""")
first_seen_in_p2p = pd.DataFrame(
    connection.execute(first_seen_in_p2p_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'block_root', 'first_seen_in_p2p']
)

print("Getting first seen attestation for block in p2p data...")
first_seen_attestation_in_p2p_query = text("""
    SELECT
        slot,
        beacon_block_root as block_root,
        MIN(propagation_slot_start_diff) as first_seen_attestation_in_p2p
    FROM libp2p_gossipsub_beacon_attestation FINAL
    WHERE
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
    GROUP BY slot, beacon_block_root
""")
first_seen_attestation_in_p2p = pd.DataFrame(
    connection.execute(first_seen_attestation_in_p2p_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'block_root', 'first_seen_attestation_in_p2p']
)

print("Getting arrival times...")
aggregated_arrival_times_query = text("""
    WITH arrival_times AS (
        SELECT
            slot,
            slot_start_date_time,
            meta_client_name,
            propagation_slot_start_diff
        FROM beacon_api_eth_v1_events_head FINAL
        WHERE
            slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
            AND meta_client_name != ''
            AND meta_client_name IS NOT NULL
            AND meta_network_name = 'mainnet'
            AND propagation_slot_start_diff < 6000
    )
    SELECT
        slot,
        CASE
            WHEN meta_client_name NOT LIKE 'ethpandaops%' THEN 'home users'
            ELSE 'ethpandaops'
        END as observed_by_group,
        min(propagation_slot_start_diff) as min_arrival_time,
        quantile(0.50)(propagation_slot_start_diff) as p50_arrival_time,
        quantile(0.90)(propagation_slot_start_diff) as p90_arrival_time,
        quantile(0.99)(propagation_slot_start_diff) as p99_arrival_time
    FROM arrival_times
    GROUP BY 
        slot,
        CASE
            WHEN meta_client_name NOT LIKE 'ethpandaops%' THEN 'home users'
            ELSE 'ethpandaops'
        END
    ORDER BY slot
""")

aggregated_arrival_times = pd.DataFrame(
    connection.execute(aggregated_arrival_times_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'observed_by_group', 'min_arrival_time', 'p50_arrival_time', 'p90_arrival_time', 'p99_arrival_time']
)

# Convert blob_sidecars to a dictionary
blob_sidecars_map = blob_sidecars.set_index('slot').to_dict(orient='index')

# Build combined data using block_sizes as the canonical list of slots
combined_data = block_sizes.copy()
combined_data = combined_data.merge(proposer_entities, on='proposer_index', how='left')
# combined_data = combined_data.merge(first_seen_in_p2p, on=['slot', 'block_root'], how='left')
# combined_data = combined_data.merge(first_seen_attestation_in_p2p, on=['slot', 'block_root'], how='left')
# combined_data = combined_data.merge(arrival_times[['slot', 'min_arrival_time', 'p50_arrival_time', 'p90_arrival_time', 'p99_arrival_time']], on='slot', how='left')
combined_data['num_blobs'] = combined_data['slot'].map(lambda slot: blob_sidecars_map.get(slot, {'num_blobs': 0})['num_blobs'])
combined_data['total_blob_size'] = combined_data['slot'].map(lambda slot: blob_sidecars_map.get(slot, {'total_blob_size': 0})['total_blob_size'])

# Calculate total data per slot
combined_data['total_data_per_slot'] = combined_data['block_total_bytes_compressed'] + combined_data['total_blob_size']
combined_data['total_size_mb'] = (combined_data['block_total_bytes_compressed'] + combined_data['total_blob_size']) / 1_000_000  


combined_data['block_pct'] = combined_data['block_total_bytes_compressed'] / combined_data['total_data_per_slot'] * 100
combined_data['blob_pct'] = combined_data['total_blob_size'] / combined_data['total_data_per_slot'] * 100

# Add 'via_mev' column to combined_data
combined_data['via_mev'] = combined_data['slot'].isin(mev_slots['slot'])
combined_data['date'] = pd.to_datetime(combined_data['time']).dt.date


# Create a dataframe with slots and their sizes
mev_slots_with_sizes = combined_data[combined_data['via_mev'] == True]
non_mev_slots_with_sizes = combined_data[combined_data['via_mev'] == False]


In [7]:
# # Calculate which event happened first for each slot
# combined_data['first_event'] = combined_data.apply(lambda row: min(
#     row['first_seen_in_p2p'] or float('inf'),
#     row['first_seen_attestation_in_p2p'] or float('inf'), 
#     row['min_arrival_time'] or float('inf')
# ), axis=1)

# # Create labels for each event type
# combined_data['first_event_type'] = combined_data.apply(lambda row: 
#     'Block in P2P' if row['first_seen_in_p2p'] == row['first_event'] 
#     else 'Attestation for Block in P2P' if row['first_seen_attestation_in_p2p'] == row['first_event']
#     else 'Block Arrival', axis=1)

# # Create 15 slot buckets and get bucket ranges
# combined_data['slot_bucket'], bins = pd.qcut(combined_data['slot'], q=15, labels=False, retbins=True)
# bucket_ranges = [f'{int(bins[i])}-{int(bins[i+1])}' for i in range(len(bins)-1)]

# # Calculate percentages per bucket
# bucket_counts = combined_data.groupby(['slot_bucket', 'first_event_type']).size().unstack(fill_value=0)
# bucket_percentages = bucket_counts.div(bucket_counts.sum(axis=1), axis=0) * 100

# # Create stacked bar chart with improved styling
# plt.figure(figsize=(16, 8))
# ax = bucket_percentages.plot(
#     kind='bar', 
#     stacked=True, 
#     color=['#3498db', '#2ecc71', '#e74c3c'],
#     width=0.8
# )

# plt.title('Distribution of First Block Event Type by Slot Range', pad=20, fontsize=16, fontweight='bold')
# plt.xlabel('Slot Range', fontsize=14)
# plt.ylabel('Percentage', fontsize=14)

# # Customize legend
# plt.legend(
#     title='Event Type',
#     title_fontsize=12,
#     fontsize=10,
#     bbox_to_anchor=(1.05, 1),
#     loc='upper left',
#     borderaxespad=0
# )

# # Customize grid
# plt.grid(axis='y', linestyle='--', alpha=0.3)

# # Set x-axis labels to slot ranges
# plt.xticks(range(len(bucket_ranges)), bucket_ranges, rotation=45, ha='right')

# plt.show()

# # Print statistics
# print("\nAverage Distribution Across Buckets:")
# avg_pct = bucket_percentages.mean()
# for event_type, pct in avg_pct.items():
#     print(f"{event_type}: {pct:.1f}%")


In [8]:
# # Calculate processing delay by comparing p2p arrival vs block arrival
# processing_delay = combined_data[['slot', 'first_seen_in_p2p', 'min_arrival_time']].dropna()
# processing_delay['delay'] = processing_delay['min_arrival_time'] - processing_delay['first_seen_in_p2p']

# # Filter out negative delays
# processing_delay = processing_delay[processing_delay['delay'] >= 0]

# # Create 15 slot buckets and get bucket ranges
# processing_delay['slot_bucket'], bins = pd.qcut(processing_delay['slot'], q=15, labels=False, retbins=True)
# bucket_ranges = [f'{int(bins[i])}-{int(bins[i+1])}' for i in range(len(bins)-1)]

# # Calculate median delay and 95th/5th percentiles per bucket
# bucket_stats = pd.DataFrame({
#     'median': processing_delay.groupby('slot_bucket')['delay'].median(),
#     'p98': processing_delay.groupby('slot_bucket')['delay'].apply(lambda x: x.quantile(0.95)),
#     'p5': processing_delay.groupby('slot_bucket')['delay'].apply(lambda x: x.quantile(0.05))
# })

# # Create figure with proper size
# plt.figure(figsize=(8, 5.5))

# # Plot median and percentiles for each bucket
# plt.bar(range(len(bucket_ranges)), bucket_stats['median'], 
#         alpha=0.7, color='#3498db', label='Median delay')
# plt.plot(range(len(bucket_ranges)), bucket_stats['p98'], 
#          color='#e74c3c', label='95th percentile', linewidth=2)
# plt.plot(range(len(bucket_ranges)), bucket_stats['p5'],
#          color='#2ecc71', label='5th percentile', linewidth=2)

# # Style the plot
# plt.title('Block Processing Delay Distribution by Slot Range', pad=20, fontsize=14, fontweight='bold')
# plt.xlabel('Slot Range', fontsize=12)
# plt.ylabel('Processing Delay (ms)', fontsize=12)
# plt.grid(True, alpha=0.3, linestyle='--')
# plt.legend(fontsize=10)

# # Set x-axis labels to slot ranges
# plt.xticks(range(len(bucket_ranges)), bucket_ranges, rotation=45, ha='right')

# # Adjust layout to prevent label cutoff
# plt.tight_layout()
# plt.show()

# print(f"Overall median processing delay: {processing_delay['delay'].median():.2f}ms")
# print(f"Overall 95th percentile delay: {processing_delay['delay'].quantile(0.95):.2f}ms")
# print(f"Overall 5th percentile delay: {processing_delay['delay'].quantile(0.05):.2f}ms")


In [ ]:
# Get head arrival times per client
print("Getting head arrival times...")
head_arrival_query = text("""
    SELECT 
        slot,
        MIN(propagation_slot_start_diff) as head_arrival_time,
        meta_client_name
    FROM beacon_api_eth_v1_events_head FINAL
    WHERE 
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
    GROUP BY slot, meta_client_name
""")
head_arrival_times = pd.DataFrame(
    connection.execute(head_arrival_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'head_arrival_time', 'meta_client_name']
)


In [ ]:
# Get block arrival times per client
print("Getting block arrival times...")
block_arrival_query = text("""
    SELECT 
        slot,
        MIN(propagation_slot_start_diff) as block_arrival_time,
        meta_client_name
    FROM beacon_api_eth_v1_events_block FINAL
    WHERE 
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_network_name = 'mainnet'
    GROUP BY slot, meta_client_name
""")
block_arrival_times = pd.DataFrame(
    connection.execute(block_arrival_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'block_arrival_time', 'meta_client_name']
)


In [ ]:
# Get blob arrival times per client
print("Getting blob arrival times...")
blob_arrival_query = text("""
    SELECT
        slot,
        MAX(propagation_slot_start_diff) as last_blob_arrival_time,
        meta_client_name
    FROM beacon_api_eth_v1_events_blob_sidecar FINAL
    WHERE
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC') 
        AND meta_network_name = 'mainnet'
    GROUP BY slot, meta_client_name
""")
blob_arrival_times = pd.DataFrame(
    connection.execute(blob_arrival_query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['slot', 'last_blob_arrival_time', 'meta_client_name']
)


In [ ]:
# Merge all arrival times into one dataframe
raw_arrival_data = pd.merge(head_arrival_times, block_arrival_times, on=['slot', 'meta_client_name'], how='outer')
raw_arrival_data = pd.merge(raw_arrival_data, blob_arrival_times, on=['slot', 'meta_client_name'], how='outer')

# Get the latest arrival time between head/block/blob for each client/slot
# Handle missing columns by using fillna
raw_arrival_data['arrival_time'] = raw_arrival_data[[
    'head_arrival_time', 
    'block_arrival_time', 
    'last_blob_arrival_time'
]].fillna(-float('inf')).max(axis=1)

print(raw_arrival_data)

In [ ]:
# Add country and group info
query = text("""
    SELECT 
        meta_client_name,
        meta_client_geo_country as country,
        CASE
            WHEN meta_client_name NOT LIKE 'ethpandaops%' THEN 'home users'
            ELSE 'ethpandaops'
        END as observed_by_group
    FROM beacon_api_eth_v1_events_head FINAL
    WHERE
        slot_start_date_time BETWEEN toDateTime(:start_date, 'UTC') AND toDateTime(:end_date, 'UTC')
        AND meta_client_name != ''
        AND meta_client_name IS NOT NULL
        AND meta_network_name = 'mainnet'
    GROUP BY meta_client_name, meta_client_geo_country
""")

client_info = pd.DataFrame(
    connection.execute(query, {"start_date": start_date.replace('Z', ''), "end_date": end_date.replace('Z', '')}).fetchall(),
    columns=['meta_client_name', 'country', 'observed_by_group']
)

print("Query done.. merging everything together...")

arrival_data = pd.merge(raw_arrival_data, client_info, on='meta_client_name', how='left')

# Filter out very late arrivals and replace -inf with NaN
# Subtract 100ms to account for the 100ms delay from the event stream processing.
arrival_data['arrival_time'] = arrival_data['arrival_time'].replace(-float('inf'), float('nan')) - 100
arrival_data = arrival_data[arrival_data['arrival_time'] < 6000]

if len(arrival_data) == 0:
    raise ValueError("No arrival times found for the given time range")

combined_data = pd.merge(combined_data, arrival_data, on=['slot'], how='left')

print(f"Found arrival times for {len(arrival_data)} slots")
print(arrival_data)


In [ ]:
worst_case_block_w_eip_7623=0.72
worst_case_block_w_eip_7623_36m=0.864
worst_case_block_w_eip_7623_60m=1.43

# Calculate p99 block size from data
base_size = combined_data['block_total_bytes_compressed'].quantile(0.99) / 1_000_000  # Convert to MB

block_sizes = [
    {
        'label': 'Historical p99 block size',
        'value': base_size
    },
    {
        'label': 'Worst case with EIP7623 (30M Gas Limit)',
        'value': worst_case_block_w_eip_7623
    },
    {
        'label': 'Worst case with EIP7623 (36M Gas Limit)',
        'value': worst_case_block_w_eip_7623_36m
    },
    {
        'label': 'Worst case with EIP7623 (60M Gas Limit)',
        'value': worst_case_block_w_eip_7623_60m
    }
]


print(block_sizes)

In [15]:
entities = combined_data['entity'].value_counts().nlargest(10).index.tolist()

# entities = ['solo_stakers','kiln']


In [ ]:
all_data = (arrival_data
    .merge(combined_data[['slot', 'block_total_bytes_compressed', 'total_blob_size', 'entity']], on='slot'))

home_users_data = all_data.query("observed_by_group == 'home users'")

print(f"Slots: {len(home_users_data['slot'].unique())}")
print(f"Arrival events: {len(home_users_data)}")
print(f"Countries: {len(home_users_data['country'].unique())}")
print(f"Countries: {', '.join(sorted(home_users_data['country'].unique()))}")


In [ ]:
from matplotlib import gridspec, colors

# Define color map for title parameters
title_param_colors = {
    'mev_status': ['#2E86C1', '#E74C3C', '#27AE60'],  
    'entity_colors': ['#8E44AD', '#E67E22', '#16A085', '#2980B9', '#C0392B', '#7D3C98', '#F39C12', '#1ABC9C', '#2E86C1', '#E74C3C', '#27AE60', '#8E44AD', '#D35400', '#16A085', '#2980B9'],
    'observer_colors': ['#884EA0', '#D35400', '#1ABC9C']
}

observed_by_groups = ['home users']
figures = []
trend_data = []

# Load logos
ethpandaops_logo = plt.imread('./content/ethpandaops.png')
xatu_logo = plt.imread('./content/xatu.png')

feature_set = {'show_block_size': False, 'show_blob_count': True, 'show_trend_lines': False}

# only the top 5 entities
top_entities = combined_data['entity'].value_counts().head(5).index.tolist()

entities =['solo_stakers']

for block_size in block_sizes:
    for entity in entities:
        for is_mev in [True, False, None]:
            for observed_by_group in observed_by_groups:
                if observed_by_group == 'nan':
                    continue
                
                print(f"Creating plot for MEV: {is_mev} {block_size['label']} with {entity} and observed by {observed_by_group}")
                
                # Filter data based on conditions
                if is_mev is None:
                    slots_df = combined_data
                elif is_mev:
                    slots_df = mev_slots_with_sizes
                else:
                    slots_df = non_mev_slots_with_sizes
                    
                # Merge and filter data
                plot_data = (arrival_data
                    .merge(slots_df[['slot', 'block_total_bytes_compressed', 'total_blob_size', 'entity']], on='slot'))
                
                if observed_by_group != 'all nodes':
                    plot_data = plot_data.query(f"observed_by_group == '{observed_by_group}'")
                
                plot_data = plot_data.query(f"entity == '{entity}'")
                
                valid_mask = ~np.isnan(plot_data['arrival_time'])
                n_blocks = len(plot_data['slot'].unique())
                n_arrivals = np.sum(valid_mask)

                if n_blocks < 100:
                    print(f"Skipping {entity} observed by {observed_by_group} (MEV: {is_mev}) due to low block count: {n_blocks}")
                    continue

                if len(plot_data) == 0:
                    print(f"No data for {entity} observed by {observed_by_group}")
                    continue

                # Calculate total size in MB
                plot_data['total_size_mb'] = (plot_data['block_total_bytes_compressed'] + plot_data['total_blob_size']) / 1_000_000

                # Create figure and gridspec
                plt.style.use('seaborn-v0_8')
                fig = plt.figure(figsize=(14, 10))
                
                # Store metadata in figure for filename generation
                fig.metadata = {
                    'block_size': block_size['label'],
                    'entity': entity,
                    'is_mev': is_mev,
                    'observed_by': observed_by_group,
                    'features': '_'.join(k for k,v in feature_set.items() if v)
                }
                
                gs = gridspec.GridSpec(2, 1, height_ratios=[1, 10])
                
                # Create title axes
                title_ax = fig.add_subplot(gs[0])
                title_ax.axis('off')
                
                # Create main plot axes
                ax = fig.add_subplot(gs[1])

                # Calculate square bins based on axis limits
                x_max = 3.5  # MB
                y_max = 6500  # ms
                n_bins = 25  # Number of bins in each dimension
                
                x_bins = np.linspace(0, x_max, n_bins)
                y_bins = np.linspace(0, y_max, n_bins)
                
                heatmap, xedges, yedges = np.histogram2d(
                    plot_data['total_size_mb'],
                    plot_data['arrival_time'],
                    bins=[x_bins, y_bins]
                )
                
                # Plot heatmap with white to red colormap
                im = ax.pcolormesh(xedges, yedges, heatmap.T, 
                                 cmap='Reds', 
                                 norm=colors.LogNorm())
                cbar = fig.colorbar(im, ax=ax, label='Number of block arrival events')

                # Add count text to each cell
                for i in range(len(x_bins)-1):
                    for j in range(len(y_bins)-1):
                        if heatmap[i,j] > 0:  # Only show non-zero counts
                            ax.text(x_bins[i] + (x_bins[1]-x_bins[0])/2,
                                  y_bins[j] + (y_bins[1]-y_bins[0])/2,
                                  int(heatmap[i,j]),
                                  ha='center', va='center',
                                  color='white' if heatmap[i,j] > np.max(heatmap)/128 else 'black', # Changed from /4 to /8
                                  fontsize=8,
                                  fontweight='bold')

                # Add attestation deadline line
                ax.axhline(y=4000, color='black', linestyle='-', alpha=0.8)

                mev_text = "MEV relay blocks" if is_mev else "Locally built blocks" if is_mev is not None else "All blocks"
                
                # Add title text to title axes
                title = 'Block Arrival Distribution'
                title_ax.text(0.5, 0.6, title, ha='center', va='bottom',
                        fontsize=16, color='black', fontweight='bold')
                title_ax.text(0.5, 0.3, mev_text, ha='center', va='bottom',
                        fontsize=16, color=title_param_colors['mev_status'][0 if is_mev else 1 if is_mev is not None else 2], 
                        fontweight='bold')
                title_ax.text(0.5, 0.0, 'Blocks created by ', ha='right', va='bottom',
                        fontsize=16, color='black')
                title_ax.text(0.5, 0.0, entity, ha='left', va='bottom',
                        fontsize=16, color=title_param_colors['entity_colors'][hash(entity) % len(title_param_colors['entity_colors'])], 
                        fontweight='bold')
                title_ax.text(0.5, -0.3, 'Blocks seen by ', ha='right', va='bottom',
                        fontsize=16, color='black')
                title_ax.text(0.5, -0.3, observed_by_group, ha='left', va='bottom',
                        fontsize=16, color=title_param_colors['observer_colors'][hash(observed_by_group) % len(title_param_colors['observer_colors'])], 
                        fontweight='bold')

                # Add legend for cell values and attestation deadline
                legend_text = ('Cell values indicate number of\nblock arrival events in that region\n\n'
                             'Black line indicates attestation\ndeadline at 4s')
                ax.text(0.75, 0.95, legend_text,
                        ha='left', va='top', transform=ax.transAxes,
                        fontsize=10, color='black', style='italic',
                        bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray'))

                # Add logos and text
                logo_ax1 = fig.add_axes([0.8, 0.75, 0.1, 0.1]) 
                logo_ax1.imshow(xatu_logo)
                logo_ax1.axis('off')
                
                text_ax = fig.add_axes([0.8, 0.73, 0.1, 0.02])
                text_ax.text(0.5, 0.5, 'Data from Xatu', ha='center', va='center', fontsize=10, fontweight='bold')
                text_ax.axis('off')

                logo_ax2 = fig.add_axes([0.13, 0.75, 0.1, 0.1])
                logo_ax2.imshow(ethpandaops_logo)
                logo_ax2.axis('off')

                text_ax = fig.add_axes([0.13, 0.73, 0.1, 0.02])
                text_ax.text(0.5, 0.5, 'ethpandaops.io', ha='center', va='center', fontsize=10, fontweight='bold')
                text_ax.axis('off')

                # Add block and arrival counts
                ax.text(0.65, 0.12, f'Total Blocks: {n_blocks:,}\nArrival events: {n_arrivals:,}', 
                        ha='left', va='top', transform=ax.transAxes,
                        fontsize=12, color='black', 
                        bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray'))
                ax.set_xlabel('Block + Blob Actual Size Compressed (MB)', fontsize=12, fontweight='bold')
                ax.set_ylabel('Block Arrival Time (milliseconds)', fontsize=12, fontweight='bold')
                ax.grid(True, alpha=0.2, linestyle='--')
                ax.set_ylim(0, 6500)
                ax.set_xlim(0, 3.5)

                plt.subplots_adjust(top=0.85)
                plt.show()
                plt.close()

In [36]:
# # Create combined CDF of arrivals for all configs
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import numpy as np

# # Create single CDF plot with all configs
# fig = go.Figure()

# colors = ['#2E86C1', '#E74C3C', '#28B463', '#F39C12', '#8E44AD', '#17A2B8', '#FD7E14', '#20C997']
# color_idx = 0

# # Dedupe configs by removing feature_set from uniqueness check
# unique_configs = []
# seen_configs = set()

# for config in configs:
#     # Create a key without feature_set for deduplication
#     dedup_key = (
#         config['block_size']['label'],
#         config['entity'],
#         config['is_mev'],
#         config['observed_by_group']
#     )
    
#     if dedup_key not in seen_configs:
#         seen_configs.add(dedup_key)
#         unique_configs.append(config)

# for config in unique_configs:
#     feature_set = config['feature_set']
#     block_size = config['block_size']
#     entity = config['entity']
#     is_mev = config['is_mev']
#     observed_by_group = config['observed_by_group']
    
#     print(f"Adding CDF for MEV: {is_mev} {block_size['label']} with {entity} and observed by {observed_by_group}")
    
#     # Filter data based on conditions
#     if is_mev is None:
#         slots_df = combined_data
#     elif is_mev:
#         slots_df = mev_slots_with_sizes
#     else:
#         slots_df = non_mev_slots_with_sizes
        
#     # Merge and filter data
#     plot_data = (arrival_data
#         .merge(slots_df[['slot', 'block_total_bytes_compressed', 'total_blob_size', 'entity']], on='slot'))
    
#     if observed_by_group != 'all nodes':
#         plot_data = plot_data.query(f"observed_by_group == '{observed_by_group}'")
    
#     plot_data = plot_data.query(f"entity == '{entity}'")
    
#     valid_mask = ~np.isnan(plot_data['arrival_time'])
#     plot_data = plot_data[valid_mask]
    
#     n_blocks = len(plot_data['slot'].unique())
#     n_arrivals = len(plot_data)

#     if n_blocks < 100:
#         print(f"Skipping {entity} observed by {observed_by_group} (MEV: {is_mev}) due to low block count: {n_blocks}")
#         continue

#     if len(plot_data) == 0:
#         print(f"No data for {entity} observed by {observed_by_group}")
#         continue

#     # Get arrival times for CDF
#     arrival_times = plot_data['arrival_time'].values
    
#     # Calculate CDF values using normalized percentiles for faster computation
#     percentiles = np.linspace(0, 100, min(1000, len(arrival_times)))
#     x_values = np.percentile(arrival_times, percentiles)
#     y_values = percentiles / 100
    
#     # Create trace name
#     mev_text = "MEV relay" if is_mev else "Locally built" if is_mev is not None else "All blocks"
#     trace_name = f"{mev_text} - {entity} - {observed_by_group}"
    
#     # Add CDF line
#     fig.add_trace(go.Scatter(
#         x=x_values,
#         y=y_values,
#         mode='lines',
#         name=trace_name,
#         line=dict(color=colors[color_idx % len(colors)], width=2)
#     ))
    
#     color_idx += 1

# # Add vertical line at 4000ms (attestation deadline)
# fig.add_vline(
#     x=4000,
#     line_dash="dash",
#     line_color="red",
#     annotation_text="Attestation deadline (4s)",
#     annotation_position="top right"
# )

# # Update layout
# fig.update_layout(
#     title="CDF of Block Arrival Times - All Configurations",
#     xaxis_title="Block Arrival Time (milliseconds)",
#     yaxis_title="Cumulative Probability",
#     width=1000,
#     height=700,
#     showlegend=True,
#     template="plotly_white",
#     legend=dict(
#         orientation="v",
#         yanchor="top",
#         y=1,
#         xanchor="left",
#         x=1.02
#     )
# )

# fig.show()


# P66 Head Time Analysis

Analyze when 66% of nodes have seen each block (p66 head time) - this represents when a block can be accepted by the network.

In [ ]:
# Calculate p66 head time for each slot across different node groups
# Function to calculate p66 for a group of arrival times
def calculate_p66(arrival_times):
    if len(arrival_times) == 0:
        return np.nan
    return np.percentile(arrival_times, 66)

# Calculate p66 for each slot and node group
p66_results = []

for slot in arrival_data['slot'].unique():
    slot_data = arrival_data[arrival_data['slot'] == slot]
    
    # Skip slots with too few observations
    if len(slot_data) < 10:
        continue
    
    # Get slot metadata from combined_data
    slot_metadata_df = combined_data[combined_data['slot'] == slot]
    
    # Skip if slot not found in combined_data
    if len(slot_metadata_df) == 0:
        continue
        
    slot_metadata = slot_metadata_df.iloc[0]
    
    # Calculate p66 for all nodes
    all_nodes_p66 = calculate_p66(slot_data['arrival_time'].dropna())
    
    # Calculate p66 for ethpandaops nodes only
    ethpandaops_data = slot_data[slot_data['observed_by_group'] == 'ethpandaops']
    ethpandaops_p66 = calculate_p66(ethpandaops_data['arrival_time'].dropna())
    
    # Calculate p66 for home users only
    home_users_data = slot_data[slot_data['observed_by_group'] == 'home users']
    home_users_p66 = calculate_p66(home_users_data['arrival_time'].dropna())
    
    # Store results
    p66_results.append({
        'slot': slot,
        'proposer_index': slot_metadata['proposer_index'],
        'entity': slot_metadata['entity'],
        'via_mev': slot_metadata['via_mev'],
        'block_size_mb': slot_metadata['block_total_bytes_compressed'] / 1_000_000,
        'total_size_mb': slot_metadata['total_size_mb'],
        'num_blobs': slot_metadata['num_blobs'],
        'p66_all': all_nodes_p66,
        'p66_ethpandaops': ethpandaops_p66,
        'p66_homeusers': home_users_p66
    })

# Create DataFrame
p66_df = pd.DataFrame(p66_results)

In [ ]:
# Create interactive CDF plot of p66 head times with Plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from PIL import Image
import base64
from io import BytesIO

# Create figure
fig = go.Figure()

# Color scheme
colors = {
    'all': '#2E86C1',
    'ethpandaops': '#E74C3C', 
    'homeusers': '#28B463'
}

# Nice names for display
nice_names = {
    'all': 'All nodes',
    'ethpandaops': 'ethPandaOps',
    'homeusers': 'Home users'
}

# Line styles for MEV vs non-MEV
dash_styles = {
    True: 'solid',  # MEV blocks
    False: 'dash'  # Non-MEV blocks
}

# Plot CDFs for different combinations
for node_group, color in colors.items():
    for is_mev in [True, False]:
        # Get the appropriate p66 column
        p66_col = f'p66_{node_group}'
        
        # Filter data
        data = p66_df[p66_df['via_mev'] == is_mev][p66_col].dropna()
        
        if len(data) > 0:
            # Sort values for CDF
            sorted_data = np.sort(data)
            y_values = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
            
            # Calculate statistics
            p98 = np.percentile(data, 98)
            p95 = np.percentile(data, 95)
            median = np.percentile(data, 50)
            
            # Create label with p99
            mev_label = 'MEV' if is_mev else 'Local'
            label = f'{mev_label} blocks seen by {nice_names[node_group]} (p98: {p98:.0f}ms)'
            
            # Add trace
            fig.add_trace(go.Scatter(
                x=sorted_data,
                y=y_values,
                mode='lines',
                name=label,
                line=dict(
                    color=color,
                    width=2.5,
                    dash=dash_styles[is_mev]
                ),
                hovertemplate=f'<b>{nice_names[node_group]} - {mev_label}</b><br>' +
                             'P66 Time: %{x:.0f}ms<br>' +
                             'Percentile: %{y:.1%}<br>' +
                             f'<b>Stats:</b><br>' +
                             f'Median: {median:.0f}ms<br>' +
                             f'P95: {p95:.0f}ms<br>' +
                             f'P98: {p98:.0f}ms' +
                             '<extra></extra>'
            ))

# Add vertical line at 4000ms (attestation deadline)
fig.add_vline(
    x=4000,
    line_dash="dot",
    line_color="red",
    line_width=2,
    annotation_text="Attestation deadline (4s)",
    annotation_position="top",
    annotation_font_size=12,
    annotation_yshift=-150  # Move text to middle of plot
)

# Calculate overall statistics for annotation
stats_text = "<b>Overall Statistics:</b><br>"
for node_group in ['all', 'ethpandaops', 'homeusers']:
    p66_col = f'p66_{node_group}'
    median = p66_df[p66_col].median()
    p95 = p66_df[p66_col].quantile(0.95)
    pct_under_4s = (p66_df[p66_col] < 4000).sum() / len(p66_df[p66_col].dropna()) * 100
    stats_text += f"{nice_names[node_group]}: median={median:.0f}ms, p95={p95:.0f}ms, &lt;4s={pct_under_4s:.1f}%<br>"

# Load logos and convert to base64
def image_to_base64(image_path):
    img = Image.open(image_path)
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return f"data:image/png;base64,{img_str}"

ethpandaops_b64 = image_to_base64('./content/ethpandaops.png')
xatu_b64 = image_to_base64('./content/xatu.png')

# Update layout with logos
fig.update_layout(
    title={
        'text': 'CDF of P66 Head Times by Node Group and Block Type',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial Black'}
    },
    xaxis=dict(
        title="P66 Head Time (milliseconds)",
        titlefont=dict(size=14, family='Arial Black'),
        range=[0, 6000],
        gridcolor='rgba(128,128,128,0.2)',
        showgrid=True,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        ticks='outside',
        tickmode='linear',
        tick0=0,
        dtick=1000
    ),
    yaxis=dict(
        title="Cumulative Probability",
        titlefont=dict(size=14, family='Arial Black'),
        range=[0, 1],
        gridcolor='rgba(128,128,128,0.2)',
        showgrid=True,
        tickformat='.0%',
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=False,
        ticks='outside',
        tickmode='linear',
        tick0=0,
        dtick=0.2
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    width=1200,
    height=700,
    hovermode='x unified',
    legend=dict(
        x=0.98,
        y=0.02,
        xanchor='right',
        yanchor='bottom',
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='rgba(0,0,0,0.2)',
        borderwidth=1,
        font=dict(size=11)
    ),
    margin=dict(t=120)  # Add more top margin for logos
)

# Add statistics annotation
fig.add_annotation(
    text=stats_text,
    xref="paper",
    yref="paper",
    x=0.02,
    y=0.98,
    xanchor="left",
    yanchor="top",
    showarrow=False,
    bordercolor="rgba(0,0,0,0.2)",
    borderwidth=1,
    bgcolor="rgba(255,255,255,0.9)",
    font=dict(size=10),
    align="left"
)

# Add ethpandaops logo (bigger)
fig.add_layout_image(
    dict(
        source=ethpandaops_b64,
        xref="paper",
        yref="paper",
        x=0.08,
        y=1.15,
        sizex=0.12,  # Increased from 0.08
        sizey=0.12,  # Increased from 0.08
        xanchor="center",
        yanchor="middle"
    )
)

# Add Xatu logo
fig.add_layout_image(
    dict(
        source=xatu_b64,
        xref="paper",
        yref="paper",
        x=0.92,
        y=1.15,
        sizex=0.08,
        sizey=0.08,
        xanchor="center",
        yanchor="middle"
    )
)

# Add text under logos
fig.add_annotation(
    text="<b>ethpandaops.io</b>",
    xref="paper",
    yref="paper",
    x=0.08,
    y=1.06,
    xanchor="center",
    yanchor="top",
    showarrow=False,
    font=dict(size=11, color='#2C3E50')
)

fig.add_annotation(
    text="<b>Data from Xatu</b>",
    xref="paper",
    yref="paper",
    x=0.92,
    y=1.08,
    xanchor="center",
    yanchor="top",
    showarrow=False,
    font=dict(size=11, color='#2C3E50')
)

# Show the plot
fig.show()

In [ ]:
# Analyze p66 times by entity with branding
# Get top entities by block count
top_entities = p66_df['entity'].value_counts().head(10).index.tolist()

# Create figure with subplots
plt.style.use('seaborn-v0_8')
fig = plt.figure(figsize=(18, 10))

# Create grid with space for title and logos
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 10], hspace=0.05, figure=fig)

# Create title/logo axes
title_ax = fig.add_subplot(gs[0])
title_ax.axis('off')

# Create subplot axes for the charts
chart_ax = fig.add_subplot(gs[1])
chart_ax.axis('off')

# Now create subplots within the chart area
subfigs = fig.subfigures(1, 1, wspace=0.1, height_ratios=[1])
subfig = subfigs
axes = subfig.subplots(1, 3)

node_groups = ['all', 'ethpandaops', 'homeusers']
titles = ['All Nodes', 'EthPandaOps Nodes', 'Home Users']

for idx, (node_group, title) in enumerate(zip(node_groups, titles)):
    ax = axes[idx]
    p66_col = f'p66_{node_group}'
    
    # Calculate median p66 for each entity
    entity_stats = []
    for entity in top_entities:
        entity_data = p66_df[p66_df['entity'] == entity][p66_col].dropna()
        if len(entity_data) > 0:
            entity_stats.append({
                'entity': entity,
                'median_p66': entity_data.median(),
                'p25': entity_data.quantile(0.25),
                'p75': entity_data.quantile(0.75),
                'count': len(entity_data)
            })
    
    entity_stats_df = pd.DataFrame(entity_stats).sort_values('median_p66')
    
    # Create bar plot with error bars
    positions = range(len(entity_stats_df))
    ax.bar(positions, entity_stats_df['median_p66'], 
           yerr=[entity_stats_df['median_p66'] - entity_stats_df['p25'],
                 entity_stats_df['p75'] - entity_stats_df['median_p66']],
           capsize=5, alpha=0.7, color='skyblue', edgecolor='navy')
    
    # Add attestation deadline line
    ax.axhline(y=4000, color='red', linestyle='--', alpha=0.7, 
               label='Attestation deadline')
    
    # Styling
    ax.set_xlabel('Entity', fontsize=12, fontweight='bold')
    ax.set_ylabel('P66 Head Time (ms)', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14)
    ax.set_xticks(positions)
    ax.set_xticklabels(entity_stats_df['entity'], rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add block counts as text
    for i, (_, row) in enumerate(entity_stats_df.iterrows()):
        ax.text(i, row['median_p66'] + 50, f"n={row['count']}", 
                ha='center', va='bottom', fontsize=8)

# Add title to title axes
title_ax.text(0.5, 0.5, 'P66 Head Times by Entity', 
              ha='center', va='center', fontsize=16, fontweight='bold')

# Load and add logos
ethpandaops_logo = plt.imread('./content/ethpandaops.png')
xatu_logo = plt.imread('./content/xatu.png')

# Add logos to title area
logo_size = 0.06
# Ethpandaops logo on left
logo_ax1 = fig.add_axes([0.05, 0.91, logo_size, logo_size])
logo_ax1.imshow(ethpandaops_logo)
logo_ax1.axis('off')

text_ax1 = fig.add_axes([0.05, 0.89, logo_size, 0.02])
text_ax1.text(0.5, 0.5, 'ethpandaops.io', ha='center', va='center', fontsize=10, fontweight='bold')
text_ax1.axis('off')

# Xatu logo on right
logo_ax2 = fig.add_axes([0.89, 0.91, logo_size, logo_size])
logo_ax2.imshow(xatu_logo)
logo_ax2.axis('off')

text_ax2 = fig.add_axes([0.89, 0.89, logo_size, 0.02])
text_ax2.text(0.5, 0.5, 'Data from Xatu', ha='center', va='center', fontsize=10, fontweight='bold')
text_ax2.axis('off')

plt.subplots_adjust(top=0.88)
plt.show()

In [ ]:
# Analyze relationship between block size and p66 times with branding
plt.style.use('seaborn-v0_8')
fig = plt.figure(figsize=(15, 8))

# Create grid with space for title and logos
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 10], hspace=0.05, figure=fig)

# Create title/logo axes
title_ax = fig.add_subplot(gs[0])
title_ax.axis('off')

# Create subplot area
subplot_gs = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=gs[1], wspace=0.3)

for idx, (node_group, title) in enumerate(zip(['all', 'ethpandaops', 'homeusers'], 
                                             ['All Nodes', 'EthPandaOps Nodes', 'Home Users'])):
    ax = fig.add_subplot(subplot_gs[idx])
    p66_col = f'p66_{node_group}'
    
    # Create scatter plot
    valid_data = p66_df[[p66_col, 'total_size_mb']].dropna()
    
    # Separate MEV and non-MEV blocks
    mev_data = valid_data[p66_df['via_mev'] == True]
    non_mev_data = valid_data[p66_df['via_mev'] == False]
    
    # Plot scatter points
    ax.scatter(non_mev_data['total_size_mb'], non_mev_data[p66_col], 
               alpha=0.3, s=10, color='blue', label='Local blocks')
    ax.scatter(mev_data['total_size_mb'], mev_data[p66_col], 
               alpha=0.3, s=10, color='red', label='MEV blocks')
    
    # Add trend lines
    if len(valid_data) > 0:
        z = np.polyfit(valid_data['total_size_mb'], valid_data[p66_col], 1)
        p = np.poly1d(z)
        x_trend = np.linspace(valid_data['total_size_mb'].min(), 
                             valid_data['total_size_mb'].max(), 100)
        ax.plot(x_trend, p(x_trend), "k--", alpha=0.8, linewidth=2, 
                label=f'Trend: {z[0]:.1f}ms/MB')
    
    # Add attestation deadline
    ax.axhline(y=4000, color='red', linestyle=':', alpha=0.7)
    
    # Styling
    ax.set_xlabel('Total Block + Blob Size (MB)', fontsize=12, fontweight='bold')
    ax.set_ylabel('P66 Head Time (ms)', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left', fontsize=10)
    ax.set_xlim(0, 3.5)
    ax.set_ylim(0, 6000)

# Add title to title axes
title_ax.text(0.5, 0.5, 'Relationship between Block Size and P66 Head Time', 
              ha='center', va='center', fontsize=16, fontweight='bold')

# Load and add logos
ethpandaops_logo = plt.imread('./content/ethpandaops.png')
xatu_logo = plt.imread('./content/xatu.png')

# Add logos to title area
logo_size = 0.06
# Ethpandaops logo on left
logo_ax1 = fig.add_axes([0.05, 0.90, logo_size, logo_size])
logo_ax1.imshow(ethpandaops_logo)
logo_ax1.axis('off')

text_ax1 = fig.add_axes([0.05, 0.88, logo_size, 0.02])
text_ax1.text(0.5, 0.5, 'ethpandaops.io', ha='center', va='center', fontsize=10, fontweight='bold')
text_ax1.axis('off')

# Xatu logo on right
logo_ax2 = fig.add_axes([0.89, 0.90, logo_size, logo_size])
logo_ax2.imshow(xatu_logo)
logo_ax2.axis('off')

text_ax2 = fig.add_axes([0.89, 0.88, logo_size, 0.02])
text_ax2.text(0.5, 0.5, 'Data from Xatu', ha='center', va='center', fontsize=10, fontweight='bold')
text_ax2.axis('off')

plt.subplots_adjust(top=0.87)
plt.show()

In [ ]:
# Save p66 data for future analysis
p66_df.to_csv('./blob-predictions/p66_head_times.csv', index=False)

# Summary statistics
stats_text = "Overall P66 Head Time Statistics:\n" + "="*50
for node_group in ['all', 'ethpandaops', 'homeusers']:
    p66_col = f'p66_{node_group}'
    stats_text += f"\n\n{node_group.upper()} NODES:"
    stats_text += f"\n  Median: {p66_df[p66_col].median():.0f} ms"
    stats_text += f"\n  Mean: {p66_df[p66_col].mean():.0f} ms"
    stats_text += f"\n  P25: {p66_df[p66_col].quantile(0.25):.0f} ms"
    stats_text += f"\n  P75: {p66_df[p66_col].quantile(0.75):.0f} ms"
    stats_text += f"\n  P95: {p66_df[p66_col].quantile(0.95):.0f} ms"
    stats_text += f"\n  P99: {p66_df[p66_col].quantile(0.99):.0f} ms"
    
    # Percentage meeting attestation deadline
    pct_under_4s = (p66_df[p66_col] < 4000).sum() / len(p66_df[p66_col].dropna()) * 100
    stats_text += f"\n  % under 4s deadline: {pct_under_4s:.1f}%"

# Data contributors summary
contributors_text = "\n\nData Contributors Summary:\n" + "="*50

# Count unique contributors
unique_nodes = arrival_data['meta_client_name'].nunique()
unique_countries = arrival_data['country'].nunique()

# Get country distribution
country_counts = arrival_data.groupby('country')['meta_client_name'].nunique().sort_values(ascending=False)
top_countries = country_counts.head(10)

# Count by node group
ethpandaops_nodes = arrival_data[arrival_data['observed_by_group'] == 'ethpandaops']['meta_client_name'].nunique()
home_user_nodes = arrival_data[arrival_data['observed_by_group'] == 'home users']['meta_client_name'].nunique()

# Get regional distribution (continents based on countries)
region_mapping = {
    'United States': 'North America',
    'Canada': 'North America',
    'Mexico': 'North America',
    'Germany': 'Europe',
    'France': 'Europe',
    'United Kingdom': 'Europe',
    'Italy': 'Europe',
    'Spain': 'Europe',
    'Belgium': 'Europe',
    'Austria': 'Europe',
    'Poland': 'Europe',
    'Czechia': 'Europe',
    'Croatia': 'Europe',
    'Slovenia': 'Europe',
    'Hungary': 'Europe',
    'Portugal': 'Europe',
    'Bulgaria': 'Europe',
    'Australia': 'Oceania',
    'New Zealand': 'Oceania',
    'Brazil': 'South America',
    'Singapore': 'Asia',
    'India': 'Asia',
    'Thailand': 'Asia',
    'Kuwait': 'Asia',
    'South Africa': 'Africa',
    'Nigeria': 'Africa'
}

arrival_data['region'] = arrival_data['country'].map(region_mapping)
unique_regions = arrival_data['region'].nunique()

contributors_text += f"\n\nTotal unique nodes: {unique_nodes:,}"
contributors_text += f"\n  - EthPandaOps nodes: {ethpandaops_nodes:,}"
contributors_text += f"\n  - Home user nodes: {home_user_nodes:,}"
contributors_text += f"\n\nTotal countries: {unique_countries}"
contributors_text += f"\nTotal regions: {unique_regions}"

contributors_text += f"\n\nTop 10 countries by node count:"
for country, count in top_countries.items():
    contributors_text += f"\n  - {country}: {count:,} nodes"

# Region summary
region_counts = arrival_data.groupby('region')['meta_client_name'].nunique().sort_values(ascending=False)
contributors_text += f"\n\nNodes by region:"
for region, count in region_counts.items():
    if pd.notna(region):
        contributors_text += f"\n  - {region}: {count:,} nodes"

# Total observations
total_observations = len(arrival_data)
contributors_text += f"\n\nTotal block arrival observations: {total_observations:,}"
contributors_text += f"\nUnique slots observed: {arrival_data['slot'].nunique():,}"

print(stats_text)
print(contributors_text)

In [128]:
# Clear and create output directory
import os
import shutil

output_dir = './blob-predictions'
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# Save each figure as PNG
for i, (size_mb, fig) in enumerate(figures):
    metadata = fig.metadata
    # Create subdirectory if it doesn't exist
    
    # Create clean filename components
    entity = metadata["entity"].lower().replace(' ', '_')
    block_size = str(metadata["block_size"]).replace('.', 'p')
    mev_flag = 'mev' if metadata["is_mev"] else 'local' if metadata["is_mev"] is not None else 'all'
    observed_by = metadata["observed_by"].lower().replace(' ', '_')
    
    clean_filename = f'block_arrival_{entity}_{block_size}mb_{mev_flag}_{observed_by}'
    filename = os.path.join(output_dir, f'{clean_filename}.png')
    
    fig.savefig(filename, bbox_inches='tight', dpi=300)
